<img src="images/cads-logo.png" style="height: 100px;" align=left> <img src="images/apache_spark.png" style="height: 20%;width:20%" align=right>

# Apache Spark DataFrames and Spark SQL

Apache Spark is a platform for distributed data processing, and it is particularly well-suited for dealing with massive data sets. The data sets that they do not readily fit within the memory or capacity of a single server.

Apache Spark has a modular architecture. A core platform is called Apache Spark core, and there are several modules, which run on top of the core platform.

In this notebook, we will mostly learn about DataFrames and work with Spark SQL. Apache Spark supports multiple languages, including:
- Scala
- Python
- Java
- Python
- R

Apache Spark's core data structure is the Resilient Distributed Dataset (RDD). RDD is a low-level object that lets Spark work by splitting data across multiple nodes in the cluster. However, working directly with RDDs is hard. Therefore, data scientists and data engineers prefer to use the Spark DataFrame abstraction built on top of RDDs.

We are particularly interested in a data structure called DataFrames. DataFrames are a set of data that are organized into columns and rows. The columns have names, and the rows have a schema. Therefore, in this way, they are very similar or analogous to tables in relational databases. Not only DataFrames are easier to understand, but also they are more optimized for complicated operations than RDDs.

There are a couple of different ways of working with DataFrames. One way is to use the DataFrame API, and basically, that is structured around using methods on DataFrame objects. The second way is Spark SQL that allows us to enter SQL queries which are executed on DataFrames, and those DataFrames are registered as tables.

### Setup Apache Spark on Jupyter
To start working with DataFrames, first of all, we have to create a `SparkSession` object from `SparkContext`. The `SparkContext` is a connection to the running cluster, and `SparkSession` is an interface with that connection.

In [1]:
#!pip install pyspark

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.getOrCreate()

The previous line of code returns an existing `SparkSession` if there's already one in the environment, or creates a new one if necessary.

In [5]:
spark

### Make the data set folder accessible

In the following cells, we are going to load a file called `location_temp.csv`, which is a time-series file which contains loacations of sensors and the temperatures taken at particular periods of time.

In [6]:
import os
MAIN_DIRECTORY = os.getcwd()

In [7]:
file_path =MAIN_DIRECTORY+"/data/location_temp.csv"

## Get Started with Spark DataFrames
To create a Spark DataFrame by loading a csv file, we can use `spark.read` function as follows.

In [8]:
df1 = spark.read.format("csv").option("header","true").load(file_path)

We can use `head(n)` method to show the heading of this data frame. `n` is the number of rows and its default value is 1.

In [9]:
df1.head(5)

[Row(event_date='03/04/2019 19:48:06', location_id='loc0', temp_celcius='29'),
 Row(event_date='03/04/2019 19:53:06', location_id='loc0', temp_celcius='27'),
 Row(event_date='03/04/2019 19:58:06', location_id='loc0', temp_celcius='28'),
 Row(event_date='03/04/2019 20:03:06', location_id='loc0', temp_celcius='30'),
 Row(event_date='03/04/2019 20:08:06', location_id='loc0', temp_celcius='27')]

If we want to show the data in a tabular format, we can use `.show(n)` method. `n` is the number of rows and its default value is 20.

In [10]:
df1.show()

+-------------------+-----------+------------+
|         event_date|location_id|temp_celcius|
+-------------------+-----------+------------+
|03/04/2019 19:48:06|       loc0|          29|
|03/04/2019 19:53:06|       loc0|          27|
|03/04/2019 19:58:06|       loc0|          28|
|03/04/2019 20:03:06|       loc0|          30|
|03/04/2019 20:08:06|       loc0|          27|
|03/04/2019 20:13:06|       loc0|          27|
|03/04/2019 20:18:06|       loc0|          27|
|03/04/2019 20:23:06|       loc0|          29|
|03/04/2019 20:28:06|       loc0|          32|
|03/04/2019 20:33:06|       loc0|          35|
|03/04/2019 20:38:06|       loc0|          32|
|03/04/2019 20:43:06|       loc0|          28|
|03/04/2019 20:48:06|       loc0|          28|
|03/04/2019 20:53:06|       loc0|          32|
|03/04/2019 20:58:06|       loc0|          34|
|03/04/2019 21:03:06|       loc0|          33|
|03/04/2019 21:08:06|       loc0|          27|
|03/04/2019 21:13:06|       loc0|          28|
|03/04/2019 2

To know the number of rows in the DataFrame, there is a useful method called `count()` that performs a count on the rows in a DataFrame.

In [11]:
df1.count()

500000

One of the useful methods in DataFrame API is `printSchema()` that prints out the schema in the tree format.

In [12]:
df1.printSchema()

root
 |-- event_date: string (nullable = true)
 |-- location_id: string (nullable = true)
 |-- temp_celcius: string (nullable = true)



### Rename column names

Now, let's load another file. In the data folder, we have another file called `utilization.csv`. This file does not have a header row. If we want to use the csv file schema, Spark provides an option to infer the columns' data types automatically. The following cells show how we can work with this type of csv file.

In [13]:
file_path =MAIN_DIRECTORY+"/data/utilization.csv"

In [14]:
df2 = spark.read.format("csv").option("header","false").option("inferSchema","true").load(file_path)

In [15]:
df2.count()

500000

In [16]:
df2.head(5)

[Row(_c0='03/05/2019 08:06:14', _c1=100, _c2=0.57, _c3=0.51, _c4=47),
 Row(_c0='03/05/2019 08:11:14', _c1=100, _c2=0.47, _c3=0.62, _c4=43),
 Row(_c0='03/05/2019 08:16:14', _c1=100, _c2=0.56, _c3=0.57, _c4=62),
 Row(_c0='03/05/2019 08:21:14', _c1=100, _c2=0.57, _c3=0.56, _c4=50),
 Row(_c0='03/05/2019 08:26:14', _c1=100, _c2=0.35, _c3=0.46, _c4=43)]

As you can see, we have five rows, but we do not have column names. Because we did not specify a header. So Spark just created column names. Basically used a pattern `_c#`.

Spark allows us to rename the columns. By using `withColumnRenamed()` method.

In [17]:
df2 = df2.withColumnRenamed('_c0', 'event_datetime')\
         .withColumnRenamed('_c1','server_id')\
         .withColumnRenamed('_c2','cpu_utilization')\
         .withColumnRenamed('_c3','free_memory')\
         .withColumnRenamed('_c4','session_count')

Here is the new DataFrame in the tabular format.

In [18]:
df2.show()

+-------------------+---------+---------------+-----------+-------------+
|     event_datetime|server_id|cpu_utilization|free_memory|session_count|
+-------------------+---------+---------------+-----------+-------------+
|03/05/2019 08:06:14|      100|           0.57|       0.51|           47|
|03/05/2019 08:11:14|      100|           0.47|       0.62|           43|
|03/05/2019 08:16:14|      100|           0.56|       0.57|           62|
|03/05/2019 08:21:14|      100|           0.57|       0.56|           50|
|03/05/2019 08:26:14|      100|           0.35|       0.46|           43|
|03/05/2019 08:31:14|      100|           0.41|       0.58|           48|
|03/05/2019 08:36:14|      100|           0.57|       0.35|           58|
|03/05/2019 08:41:14|      100|           0.41|        0.4|           58|
|03/05/2019 08:46:14|      100|           0.53|       0.35|           62|
|03/05/2019 08:51:14|      100|           0.51|        0.6|           45|
|03/05/2019 08:56:14|      100|       

In [19]:
df2.printSchema()

root
 |-- event_datetime: string (nullable = true)
 |-- server_id: integer (nullable = true)
 |-- cpu_utilization: double (nullable = true)
 |-- free_memory: double (nullable = true)
 |-- session_count: integer (nullable = true)



Another useful method in DataFrame API is `describe()` that computes basic statistics for numeric and string columns.

This include count, mean, stddev, min, and max.

In [20]:
df2.describe('cpu_utilization').show()

+-------+-------------------+
|summary|    cpu_utilization|
+-------+-------------------+
|  count|             500000|
|   mean| 0.6205177399999797|
| stddev|0.15875173872913106|
|    min|               0.22|
|    max|                1.0|
+-------+-------------------+



If no columns are given, this function computes statistics for all numerical or string columns.

In [21]:
df2.describe().show()

+-------+-------------------+------------------+-------------------+-------------------+------------------+
|summary|     event_datetime|         server_id|    cpu_utilization|        free_memory|     session_count|
+-------+-------------------+------------------+-------------------+-------------------+------------------+
|  count|             500000|            500000|             500000|             500000|            500000|
|   mean|               NULL|             124.5| 0.6205177399999797|0.37912809999999125|          69.59616|
| stddev|               NULL|14.430884120552715|0.15875173872913106|0.15830931278376184|14.850676696352798|
|    min|03/05/2019 08:06:14|               100|               0.22|                0.0|                32|
|    max|04/09/2019 01:22:46|               149|                1.0|               0.78|               105|
+-------+-------------------+------------------+-------------------+-------------------+------------------+



In [22]:
df2.describe(['free_memory','session_count']).show()

+-------+-------------------+------------------+
|summary|        free_memory|     session_count|
+-------+-------------------+------------------+
|  count|             500000|            500000|
|   mean|0.37912809999999125|          69.59616|
| stddev|0.15830931278376184|14.850676696352798|
|    min|                0.0|                32|
|    max|               0.78|               105|
+-------+-------------------+------------------+



### Load a JSON file into a DataFrame
In the following cell, we are trying to load a JSON file into a DataFrame by using `spark.read` command.

In [23]:
file_path =MAIN_DIRECTORY+"/data/utilization.json"

In [24]:
df3 = spark.read.format("json").load(file_path)

In [25]:
df3.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

In [26]:
df3.count()

153279

Now, what you will notice here is we did not have to change column names.That is because in JSON, you specify key-value pairs. For example, there was a row that has `cpu_utilization` equals to 0.77, that corresponds to the first row. This row also has a key-value pair with `free_memory` equals to 0.22 and `server_id` equals to 115.

Apache Spark provides an attribute called `columns`, to show the list of a DataFrame's columns.

In [27]:
df3.columns

['cpu_utilization',
 'event_datetime',
 'free_memory',
 'server_id',
 'session_count']

Sometimes we want to work with a subset of data. For example, we have 500000 rows of data in this DataFrame. Although they are not too many rows, it may be more than you want to work with at any particular time. And you would rather work with a sample of the data. To do that, you can use `sample` command.

In [28]:
df3_sample = df3.sample(False, 0.1)

In [29]:
df3_sample.count()

15390

DataFrame API provides a method called `sort()` to sort the rows based on one or more columns.

In [30]:
df3_sort = df3.sort('server_id').show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

If we want to sort the rows based on more that one coulmn, we can specify the list of columns and sorting order by using the following syntax.

In [31]:
df3_sorted = df3.sort(['event_datetime','server_id'], ascending = [0,1]).show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|            0.6|04/09/2019 01:21:40|       0.62|      114|           44|
|           0.74|04/09/2019 01:21:38|       0.34|      113|           74|
|           0.56|04/09/2019 01:21:36|       0.09|      112|           83|
|           0.75|04/09/2019 01:21:34|       0.29|      111|           74|
|           0.36|04/09/2019 01:21:33|       0.27|      110|           55|
|           0.64|04/09/2019 01:21:31|       0.32|      109|           76|
|           0.57|04/09/2019 01:21:29|       0.08|      108|           66|
|           0.85|04/09/2019 01:21:28|       0.46|      107|           81|
|           0.28|04/09/2019 01:21:26|       0.64|      106|           56|
|           0.49|04/09/2019 01:21:24|       0.69|      105|           73|
|           0.66|04/09/2019 01:21:23| 

### Filtering using DataFrame API

Now, let's take a look at how we can use DataFrame API to filter some of the rows in DataFrames.

One of the DataFrames that we have created is `df1`, which stores location ID, and temperature measurement at a particular point and time.

In [32]:
df1.show(5)

+-------------------+-----------+------------+
|         event_date|location_id|temp_celcius|
+-------------------+-----------+------------+
|03/04/2019 19:48:06|       loc0|          29|
|03/04/2019 19:53:06|       loc0|          27|
|03/04/2019 19:58:06|       loc0|          28|
|03/04/2019 20:03:06|       loc0|          30|
|03/04/2019 20:08:06|       loc0|          27|
+-------------------+-----------+------------+
only showing top 5 rows



If we want to filter rows based on their `location_id`, we can use `filter` command. `filter(condition)` filters rows using the given condition. `filter()` method essentially allows us to specify a `WHERE` clause.

In [33]:
df1.filter(df1['location_id']=='loc0').show()

+-------------------+-----------+------------+
|         event_date|location_id|temp_celcius|
+-------------------+-----------+------------+
|03/04/2019 19:48:06|       loc0|          29|
|03/04/2019 19:53:06|       loc0|          27|
|03/04/2019 19:58:06|       loc0|          28|
|03/04/2019 20:03:06|       loc0|          30|
|03/04/2019 20:08:06|       loc0|          27|
|03/04/2019 20:13:06|       loc0|          27|
|03/04/2019 20:18:06|       loc0|          27|
|03/04/2019 20:23:06|       loc0|          29|
|03/04/2019 20:28:06|       loc0|          32|
|03/04/2019 20:33:06|       loc0|          35|
|03/04/2019 20:38:06|       loc0|          32|
|03/04/2019 20:43:06|       loc0|          28|
|03/04/2019 20:48:06|       loc0|          28|
|03/04/2019 20:53:06|       loc0|          32|
|03/04/2019 20:58:06|       loc0|          34|
|03/04/2019 21:03:06|       loc0|          33|
|03/04/2019 21:08:06|       loc0|          27|
|03/04/2019 21:13:06|       loc0|          28|
|03/04/2019 2

If we want to count all the rows that are located in a specific `location_id`,we can specify the `count()` command.

In [34]:
df1.filter(df1['location_id']=='loc0').count()

1000

Sometimes we only need to list one or two columns; in this case, we can use `select()` method that projects a set of expressions and returns a new DataFrame. Let's take a look at how it works.

In [35]:
df1.select('location_id','temp_celcius').show()

+-----------+------------+
|location_id|temp_celcius|
+-----------+------------+
|       loc0|          29|
|       loc0|          27|
|       loc0|          28|
|       loc0|          30|
|       loc0|          27|
|       loc0|          27|
|       loc0|          27|
|       loc0|          29|
|       loc0|          32|
|       loc0|          35|
|       loc0|          32|
|       loc0|          28|
|       loc0|          28|
|       loc0|          32|
|       loc0|          34|
|       loc0|          33|
|       loc0|          27|
|       loc0|          28|
|       loc0|          33|
|       loc0|          28|
+-----------+------------+
only showing top 20 rows



### Aggregation using DataFrame API

Now, let's take a look at aggregating using the DataFrame API. In the following cell we will use `groupBy` method that groups the DataFrame using the specified columns, so we can run aggregation on them.

In [36]:
df1.groupBy('location_id').count().show()

+-----------+-----+
|location_id|count|
+-----------+-----+
|     loc196| 1000|
|     loc226| 1000|
|     loc150| 1000|
|     loc292| 1000|
|     loc311| 1000|
|      loc22| 1000|
|      loc31| 1000|
|     loc305| 1000|
|      loc82| 1000|
|      loc90| 1000|
|     loc118| 1000|
|     loc195| 1000|
|     loc208| 1000|
|      loc39| 1000|
|      loc75| 1000|
|     loc228| 1000|
|     loc203| 1000|
|     loc193| 1000|
|     loc122| 1000|
|     loc145| 1000|
+-----------+-----+
only showing top 20 rows



If we want to sort the DataFrame, we can use `orderBy` that returns a new DataFrame sorted by the specified column(s).

In [37]:
df1.orderBy('location_id').show()

+-------------------+-----------+------------+
|         event_date|location_id|temp_celcius|
+-------------------+-----------+------------+
|03/04/2019 19:48:06|       loc0|          29|
|03/04/2019 19:53:06|       loc0|          27|
|03/04/2019 19:58:06|       loc0|          28|
|03/04/2019 20:03:06|       loc0|          30|
|03/04/2019 20:08:06|       loc0|          27|
|03/04/2019 20:13:06|       loc0|          27|
|03/04/2019 20:18:06|       loc0|          27|
|03/04/2019 20:23:06|       loc0|          29|
|03/04/2019 20:28:06|       loc0|          32|
|03/04/2019 20:33:06|       loc0|          35|
|03/04/2019 20:38:06|       loc0|          32|
|03/04/2019 20:43:06|       loc0|          28|
|03/04/2019 20:48:06|       loc0|          28|
|03/04/2019 20:53:06|       loc0|          32|
|03/04/2019 20:58:06|       loc0|          34|
|03/04/2019 21:03:06|       loc0|          33|
|03/04/2019 21:08:06|       loc0|          27|
|03/04/2019 21:13:06|       loc0|          28|
|03/04/2019 2

To calculate the average temperature at each location, we can use `agg` operation. Let's take a look at the following example.

In [38]:
df1.groupBy('location_id').agg({'temp_celcius':'mean'}).orderBy('location_id').show()

+-----------+-----------------+
|location_id|avg(temp_celcius)|
+-----------+-----------------+
|       loc0|           29.176|
|       loc1|           28.246|
|      loc10|           25.337|
|     loc100|           27.297|
|     loc101|           25.317|
|     loc102|           30.327|
|     loc103|           25.341|
|     loc104|           26.204|
|     loc105|           26.217|
|     loc106|           27.201|
|     loc107|           33.268|
|     loc108|           32.195|
|     loc109|           24.138|
|      loc11|           25.308|
|     loc110|           26.239|
|     loc111|           31.391|
|     loc112|           33.359|
|     loc113|           30.345|
|     loc114|           29.261|
|     loc115|           23.239|
+-----------+-----------------+
only showing top 20 rows



There are different aggregation function options, for example, if we want to have the maximum temperature in each location, we can write the following code.

In [39]:
df1.groupBy('location_id').agg({'temp_celcius':'max'}).orderBy('location_id', ascending=False).show()

+-----------+-----------------+
|location_id|max(temp_celcius)|
+-----------+-----------------+
|      loc99|               40|
|      loc98|               39|
|      loc97|               38|
|      loc96|               35|
|      loc95|               40|
|      loc94|               32|
|      loc93|               31|
|      loc92|               40|
|      loc91|               37|
|      loc90|               30|
|       loc9|               39|
|      loc89|               37|
|      loc88|               32|
|      loc87|               38|
|      loc86|               30|
|      loc85|               35|
|      loc84|               33|
|      loc83|               33|
|      loc82|               34|
|      loc81|               30|
+-----------+-----------------+
only showing top 20 rows



### Data Sampling

Sometimes, we may want to use sampling, particularly when we have large data sets, and we are doing kind of an exploratory analysis. We want to get kind of an understanding at a high level of what the data is like. Sampling can be beneficial for doing quick operations. Now, let's see how we can take a sample, or a subset of that, but randomly. In PySpark, `sample()` method returns a sampled subset of this DataFrame, and it usually takes two parameters, `fraction` that specifies the fraction of rows to generate, range [0.0, 1.0]. The second parameter is `withReplacement`, which is a boolean parameter. Usually, we assign `false` to it, in this case, what that means is each time we pull a row out of our sampling, we don't put it back in, so we will never get duplicates, we will always get unique values.

In [40]:
df1_sample1 = df1.sample(fraction=0.1, withReplacement=False)

In [41]:
df1_sample1.count()

50454

Now, let's run some simple descriptive statistics on our sample.

In [42]:
df1_sample1.groupBy('location_id').agg({'temp_celcius':'mean'}).orderBy('location_id').show(10)

+-----------+------------------+
|location_id| avg(temp_celcius)|
+-----------+------------------+
|       loc0|29.063157894736843|
|       loc1|28.666666666666668|
|      loc10| 25.45945945945946|
|     loc100|27.060606060606062|
|     loc101|25.147368421052633|
|     loc102|              30.4|
|     loc103|25.409523809523808|
|     loc104|26.021052631578947|
|     loc105| 26.05217391304348|
|     loc106|27.413793103448278|
+-----------+------------------+
only showing top 10 rows



Now, let's compare these results to results of the original data set, the DataFrame `df1`, which has 500000 rows.

In [43]:
df1.groupBy('location_id').agg({'temp_celcius':'mean'}).orderBy('location_id').show(10)

+-----------+-----------------+
|location_id|avg(temp_celcius)|
+-----------+-----------------+
|       loc0|           29.176|
|       loc1|           28.246|
|      loc10|           25.337|
|     loc100|           27.297|
|     loc101|           25.317|
|     loc102|           30.327|
|     loc103|           25.341|
|     loc104|           26.204|
|     loc105|           26.217|
|     loc106|           27.201|
+-----------+-----------------+
only showing top 10 rows



As you can see, when we did the sampling and took 10% when we took the average of location zero, we got something that was about 29.4, but the actual is approximately 29.18. Therefore, we can see by sampling, we get very close to what the average is for the actual population. One of the things to consider is the size of the sample that we are drawing.

### Save Data from DataFrame

Sometimes after we have been working with DataFrames and creating new DataFrames and running calculations and doing sampling, we might want to save our results out. To do this, we can use `write` object and specify the `csv()` method within that, and then specify a name or what we'd like to save. It saves the DataFrame to disk using the csv format.

In [44]:
df1.write.csv('df1.csv')

Now, let's take a look at the current directory

In [45]:
! ls df1.csv # in windows, you would use dir command

part-00000-4d4e9208-5830-4e4f-8d8d-c3dece1935f9-c000.csv  _SUCCESS
part-00001-4d4e9208-5830-4e4f-8d8d-c3dece1935f9-c000.csv


Now, what you will notice here is that `df1.csv` is not a single file. It is a directory. And what is in that directory is four different files with `csv` extensions, and that is because of the way Apache Spark works internally. Spark can break up DataFrames into partition subsets, and in this case, there were four partitions. Each partition has its own file. There is also a `_SUCCESS` flag that was written out. Now, let's list the contents of one of these files.

In [46]:
! head df1.csv/part-00000-5cb7c640-19b1-48b7-b3e5-ce37cca11d95-c000.csv

 # for Windows, use more command

head: cannot open 'df1.csv/part-00000-5cb7c640-19b1-48b7-b3e5-ce37cca11d95-c000.csv' for reading: No such file or directory


To write the DataFrame in JSON format, you can use the following code.

In [47]:
df1.write.json('df1.json')

In [48]:
!ls df1.json

part-00000-6cf7019f-4ee8-44d5-8ced-33e7def79031-c000.json  _SUCCESS
part-00001-6cf7019f-4ee8-44d5-8ced-33e7def79031-c000.json


In [49]:
! head df1.json/part-00000-e3666b64-84b9-4168-8c6c-0ea6c727660c-c000.json

head: cannot open 'df1.json/part-00000-e3666b64-84b9-4168-8c6c-0ea6c727660c-c000.json' for reading: No such file or directory


## Querying DataFrames with SQL

Up to now, we've been using the Spark DataFrame API to work with DataFrames. Now, we're going to switch gears and we're going to work with SQL. In particular, we're going to use Spark SQL for working with DataFrames.

In this part, we will use `utilization.json` that includes cpu utilization, the amount of free memory at a particular point in time, and then the number of sessions that are currently connected to the server at the particular point in time.

In [50]:
file_path =MAIN_DIRECTORY+"/data/utilization.json"
df_util = spark.read.format('json').load(file_path)

In [53]:
file_path =MAIN_DIRECTORY+"/data/utilization.csv"
df_util = spark.read.format('csv').option('inferSchema', 'true').load(file_path)

In [54]:
df_util.show(10)

+-------------------+---+----+----+---+
|                _c0|_c1| _c2| _c3|_c4|
+-------------------+---+----+----+---+
|03/05/2019 08:06:14|100|0.57|0.51| 47|
|03/05/2019 08:11:14|100|0.47|0.62| 43|
|03/05/2019 08:16:14|100|0.56|0.57| 62|
|03/05/2019 08:21:14|100|0.57|0.56| 50|
|03/05/2019 08:26:14|100|0.35|0.46| 43|
|03/05/2019 08:31:14|100|0.41|0.58| 48|
|03/05/2019 08:36:14|100|0.57|0.35| 58|
|03/05/2019 08:41:14|100|0.41| 0.4| 58|
|03/05/2019 08:46:14|100|0.53|0.35| 62|
|03/05/2019 08:51:14|100|0.51| 0.6| 45|
+-------------------+---+----+----+---+
only showing top 10 rows



In [55]:
df_util = df_util.withColumnRenamed('_c0', 'event_datetime')\
                .withColumnRenamed('_c1', 'server_id')\
                .withColumnRenamed('_c2', 'cpu_utilization')\
                .withColumnRenamed('_c3', 'free_memory')\
                .withColumnRenamed('_c4', 'session_count')

In [56]:
df_util.count()

500000

To work with SQL in Spark, we have to create a temporary view. And to do that, we specify the DataFrame, and then we call the method `createOrReplaceTempView()` and then we should specify a name for this table. Let's do it.

In [57]:
df_util.createOrReplaceTempView("utilization")

Now, we have the ability to query on a table called utilization. We will create that by executing a SQL command in the Spark session.

In [58]:
df_sql = spark.sql("SELECT * FROM utilization LIMIT 10")

In [59]:
df_sql.show(20)

+-------------------+---------+---------------+-----------+-------------+
|     event_datetime|server_id|cpu_utilization|free_memory|session_count|
+-------------------+---------+---------------+-----------+-------------+
|03/05/2019 08:06:14|      100|           0.57|       0.51|           47|
|03/05/2019 08:11:14|      100|           0.47|       0.62|           43|
|03/05/2019 08:16:14|      100|           0.56|       0.57|           62|
|03/05/2019 08:21:14|      100|           0.57|       0.56|           50|
|03/05/2019 08:26:14|      100|           0.35|       0.46|           43|
|03/05/2019 08:31:14|      100|           0.41|       0.58|           48|
|03/05/2019 08:36:14|      100|           0.57|       0.35|           58|
|03/05/2019 08:41:14|      100|           0.41|        0.4|           58|
|03/05/2019 08:46:14|      100|           0.53|       0.35|           62|
|03/05/2019 08:51:14|      100|           0.51|        0.6|           45|
+-------------------+---------+-------

If we want to project on specific columns, we can do it in the following way.

In [60]:
spark.sql("SELECT server_id, free_memory FROM utilization LIMIT 10").show()

+---------+-----------+
|server_id|free_memory|
+---------+-----------+
|      100|       0.51|
|      100|       0.62|
|      100|       0.57|
|      100|       0.56|
|      100|       0.46|
|      100|       0.58|
|      100|       0.35|
|      100|        0.4|
|      100|       0.35|
|      100|        0.6|
+---------+-----------+



### Filtering DataFrames with SQL
Next, we are going to take a look at how to filter DataFrames using Spark SQL.  

In [61]:
# Example 1
spark.sql("SELECT * FROM utilization WHERE server_id = 149 LIMIT 10").show()

+-------------------+---------+---------------+-----------+-------------+
|     event_datetime|server_id|cpu_utilization|free_memory|session_count|
+-------------------+---------+---------------+-----------+-------------+
|03/05/2019 08:07:44|      149|           0.74|       0.27|           66|
|03/05/2019 08:12:44|      149|            0.9|       0.34|           85|
|03/05/2019 08:17:44|      149|           0.59|       0.19|           84|
|03/05/2019 08:22:44|      149|            0.6|       0.08|           81|
|03/05/2019 08:27:44|      149|           0.83|       0.42|           73|
|03/05/2019 08:32:44|      149|           0.75|       0.19|           84|
|03/05/2019 08:37:44|      149|            0.9|       0.14|           92|
|03/05/2019 08:42:44|      149|           0.67|       0.16|           88|
|03/05/2019 08:47:44|      149|           0.91|       0.31|           71|
|03/05/2019 08:52:44|      149|           0.82|       0.13|           72|
+-------------------+---------+-------

In [62]:
# Example 2
spark.sql("SELECT server_id as sid, session_count as sc \
            FROM utilization WHERE session_count >50 LIMIT 10").show()

+---+---+
|sid| sc|
+---+---+
|100| 62|
|100| 58|
|100| 58|
|100| 62|
|100| 60|
|100| 57|
|100| 66|
|100| 65|
|100| 66|
|100| 63|
+---+---+



In [63]:
# Example 3
spark.sql("SELECT server_id, session_count FROM utilization \
           WHERE session_count >50 AND server_id = 120 \
           ORDER BY session_count DESC \
           LIMIT 10").show()

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
+---------+-------------+



### Aggregation DataFrames with SQL

When we work with SQL in databases, we often use SQL to perform aggregations and the same holds true when working with SQL in Spark. Let's write some basic queries against the DataFrame and do a very simple aggregations.

In [64]:
# Example 1
spark.sql("SELECT count(*) FROM utilization").show()

+--------+
|count(1)|
+--------+
|  500000|
+--------+



In [65]:
# Example 2
spark.sql("SELECT count(*) FROM utilization WHERE session_count > 70").show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [66]:
# Example 3
spark.sql("SELECT server_id, count(*) FROM utilization \
           WHERE session_count > 70 \
           GROUP BY server_id").show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      108|    8375|
|      101|    9808|
|      115|    5284|
|      126|    6365|
|      103|    8744|
|      128|    3719|
|      122|    4885|
|      111|    3093|
|      120|    2733|
|      117|    3605|
|      112|    7425|
|      127|    5961|
|      107|    5646|
|      114|    2128|
|      100|     391|
|      130|    2891|
|      129|    3222|
|      102|    8586|
|      113|    9418|
|      121|    7084|
+---------+--------+
only showing top 20 rows



In [67]:
# Example 4
spark.sql("SELECT server_id, count(*) FROM utilization \
           WHERE session_count > 70 \
           GROUP BY server_id \
           ORDER BY count(*) DESC").show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      101|    9808|
|      113|    9418|
|      145|    9304|
|      103|    8744|
|      102|    8586|
|      133|    8583|
|      108|    8375|
|      149|    8288|
|      137|    8248|
|      148|    8027|
|      123|    7918|
|      118|    7913|
|      112|    7425|
|      139|    7383|
|      104|    7366|
|      121|    7084|
|      142|    7084|
|      146|    7072|
|      126|    6365|
|      144|    6220|
+---------+--------+
only showing top 20 rows



In [68]:
# Example 5
spark.sql("SELECT server_id, min(session_count), max(session_count), \
           round(avg(session_count),2) \
           FROM utilization \
           WHERE session_count > 70 \
           GROUP BY server_id \
           ORDER BY count(*) DESC").show()

+---------+------------------+------------------+----------------------------+
|server_id|min(session_count)|max(session_count)|round(avg(session_count), 2)|
+---------+------------------+------------------+----------------------------+
|      101|                71|               105|                       87.67|
|      113|                71|               103|                       86.96|
|      145|                71|               103|                       86.98|
|      103|                71|               101|                       85.76|
|      102|                71|               101|                       85.71|
|      133|                71|               100|                       85.47|
|      108|                71|               100|                       85.12|
|      149|                71|                99|                       84.96|
|      137|                71|                99|                       85.01|
|      148|                71|                99|   

### Joining DataFrames with SQL

One of the most useful features of SQL is the ability to join tables. We can join in Spark SQL as well.

First, we are going to create another temporary table based on the `server_names.csv` file.

In [70]:
file_path =MAIN_DIRECTORY+"/data/server_names.csv"
df_servers = spark.read.csv(file_path, header=True)

In [71]:
df_servers.show()

+---------+-----------+
|server_id|server_name|
+---------+-----------+
|      100| Server 100|
|      101| Server 101|
|      102| Server 102|
|      103| Server 103|
|      104| Server 104|
|      105| Server 105|
|      106| Server 106|
|      107| Server 107|
|      108| Server 108|
|      109| Server 109|
|      110| Server 110|
|      111| Server 111|
|      112| Server 112|
|      113| Server 113|
|      114| Server 114|
|      115| Server 115|
|      116| Server 116|
|      117| Server 117|
|      118| Server 118|
|      119| Server 119|
+---------+-----------+
only showing top 20 rows



In [72]:
df_servers.createOrReplaceTempView('server_name')

Now, let's quickly do a check on `server_id` in `utilization` table.

In [73]:
spark.sql('SELECT DISTINCT server_id FROM utilization ORDER BY server_id').show()

+---------+
|server_id|
+---------+
|      100|
|      101|
|      102|
|      103|
|      104|
|      105|
|      106|
|      107|
|      108|
|      109|
|      110|
|      111|
|      112|
|      113|
|      114|
|      115|
|      116|
|      117|
|      118|
|      119|
+---------+
only showing top 20 rows



Now, let's see what the minimum and maximum of server_id is.

In [74]:
spark.sql("SELECT min(server_id), max(server_id) FROM utilization").show()

+--------------+--------------+
|min(server_id)|max(server_id)|
+--------------+--------------+
|           100|           149|
+--------------+--------------+



Well, let's join these two tables.

In [75]:
df_join = spark.sql("SELECT u.server_id, sn.server_name, u.session_count \
                     FROM utilization u \
                     INNER JOIN server_name sn \
                     ON sn.server_id = u.server_id")
df_join.show()

+---------+-----------+-------------+
|server_id|server_name|session_count|
+---------+-----------+-------------+
|      100| Server 100|           47|
|      100| Server 100|           43|
|      100| Server 100|           62|
|      100| Server 100|           50|
|      100| Server 100|           43|
|      100| Server 100|           48|
|      100| Server 100|           58|
|      100| Server 100|           58|
|      100| Server 100|           62|
|      100| Server 100|           45|
|      100| Server 100|           47|
|      100| Server 100|           60|
|      100| Server 100|           57|
|      100| Server 100|           44|
|      100| Server 100|           47|
|      100| Server 100|           66|
|      100| Server 100|           65|
|      100| Server 100|           66|
|      100| Server 100|           42|
|      100| Server 100|           63|
+---------+-----------+-------------+
only showing top 20 rows



### De-Duplicating with DataFrame API

When we're working with Data Frames, Spark provides some ways to de-duplicate data. So, let's take a look at how to do that. In this part also we will learn how we can create small data sets to work within the Jupiter Notebook session. Before doing anything, please restart the Jupyter kernel.

In [76]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [77]:
sc =SparkContext.getOrCreate()

`sc` stands for `SparkContext`. It is a global variable that gives us access to the Spark Context. Here, what we want to do is create a DataFrame, and to do that, we will use `parallelize` method that creates a parallelized data structure. Spark automatically parallelize DataFrames. But now we are going to create this data manually, so we are specifying `parallelized` explicitly.

In [78]:
rdd = sc.parallelize([Row(server_name='Server 101', cpu_utilization=85, session_count=80),
                      Row(server_name='Server 101', cpu_utilization=80, session_count=90),
                      Row(server_name='Server 102', cpu_utilization=85, session_count=80),
                      Row(server_name='Server 102', cpu_utilization=85, session_count=80)])

In [79]:
spark = SparkSession(sc)

`toDF()` turns that parallelized data structure to into a DataFrame.

In [80]:
df_dup = rdd.toDF()
df_dup.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| Server 101|             85|           80|
| Server 101|             80|           90|
| Server 102|             85|           80|
| Server 102|             85|           80|
+-----------+---------------+-------------+



Now, we are going to drop duplicates. To do that we can use `drop_duplicates()` method which returns a new DataFrame with duplicate rows removed, optionally only considering certain columns.

In [81]:
  df_dup.drop_duplicates().show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| Server 101|             85|           80|
| Server 101|             80|           90|
| Server 102|             85|           80|
+-----------+---------------+-------------+



If we want to drop any time there is a duplicate in one of the columns, we can do that as well. Let's take a look at the following example.

In [82]:
 df_dup.drop_duplicates(['server_name']).show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| Server 101|             85|           80|
| Server 102|             85|           80|
+-----------+---------------+-------------+



In [83]:
 df_dup.drop_duplicates(['server_name', 'cpu_utilization']).show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| Server 101|             80|           90|
| Server 101|             85|           80|
| Server 102|             85|           80|
+-----------+---------------+-------------+



### Working with null values

It is not uncommon to have data missing from DataFrame. When we are working with SQL, we are used to work with nulls. When we working with DataFrames, the absence of data is indicated by an NA. So in this section, we are going to look how we can work with NAs and Nulls using DataFrames and Spark SQL.

In this section, we are importing a couple of things, we have not seen before. Let's take a look at them.

In [84]:
from pyspark.sql.functions import lit #allows us to create a literal column for a dataframe
from pyspark.sql.types import StringType

Now, we are going to add a column and set that column's values equall to null or NA. In this case, we will use `lit()` function that is a way for us to interact with column literals in PySpark. Spark SQL functions lit() is used to add a new column by assigning a literal or constant value to Spark DataFrame.

In [85]:
df = rdd.toDF()
df_na = df.withColumn('na_col', lit(None).cast(StringType()))

In [86]:
df_na.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| Server 101|             85|           80|  NULL|
| Server 101|             80|           90|  NULL|
| Server 102|             85|           80|  NULL|
| Server 102|             85|           80|  NULL|
+-----------+---------------+-------------+------+



Now, one of the things that we can do is globally replace all nulls or NAs with some value. And we can do that with `fillna()` function.

In [87]:
df_na.fillna('A').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| Server 101|             85|           80|     A|
| Server 101|             80|           90|     A|
| Server 102|             85|           80|     A|
| Server 102|             85|           80|     A|
+-----------+---------------+-------------+------+



Now, Let's create a DataFrame that has versions both with the nulls and with the As.

In [88]:
df_union = df_na.fillna('A').union(df_na)

In [89]:
df_union.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| Server 101|             85|           80|     A|
| Server 101|             80|           90|     A|
| Server 102|             85|           80|     A|
| Server 102|             85|           80|     A|
| Server 101|             85|           80|  NULL|
| Server 101|             80|           90|  NULL|
| Server 102|             85|           80|  NULL|
| Server 102|             85|           80|  NULL|
+-----------+---------------+-------------+------+



Now we can drop only rows with NAs in them.

In [90]:
df_union.na.drop().show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| Server 101|             85|           80|     A|
| Server 101|             80|           90|     A|
| Server 102|             85|           80|     A|
| Server 102|             85|           80|     A|
+-----------+---------------+-------------+------+



Well, let's see how we can do that with Spark SQL.

In [92]:
df_union.createOrReplaceTempView('na_table')

In [93]:
spark.sql('SELECT * FROM na_table WHERE na_col IS NULL').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| Server 101|             85|           80|  NULL|
| Server 101|             80|           90|  NULL|
| Server 102|             85|           80|  NULL|
| Server 102|             85|           80|  NULL|
+-----------+---------------+-------------+------+



## Exploratory Data Analysis with DataFrame API

DataFrame API provides some tools for some higher level tasks like exploratory data analysis. In this section, we are going to learn how to use DataFrame API for doing some basic EDA with the utilization DataFrame. First, let's take a look at this DataFrame.

In [94]:
df_util.show()

+-------------------+---------+---------------+-----------+-------------+
|     event_datetime|server_id|cpu_utilization|free_memory|session_count|
+-------------------+---------+---------------+-----------+-------------+
|03/05/2019 08:06:14|      100|           0.57|       0.51|           47|
|03/05/2019 08:11:14|      100|           0.47|       0.62|           43|
|03/05/2019 08:16:14|      100|           0.56|       0.57|           62|
|03/05/2019 08:21:14|      100|           0.57|       0.56|           50|
|03/05/2019 08:26:14|      100|           0.35|       0.46|           43|
|03/05/2019 08:31:14|      100|           0.41|       0.58|           48|
|03/05/2019 08:36:14|      100|           0.57|       0.35|           58|
|03/05/2019 08:41:14|      100|           0.41|        0.4|           58|
|03/05/2019 08:46:14|      100|           0.53|       0.35|           62|
|03/05/2019 08:51:14|      100|           0.51|        0.6|           45|
|03/05/2019 08:56:14|      100|       

In [95]:
df_util.count()

500000

One of the useful methods for doing exploratory data analysis is `.describe()`. Let's see how it works.

In [96]:
df_util.describe().show()

+-------+-------------------+------------------+-------------------+-------------------+------------------+
|summary|     event_datetime|         server_id|    cpu_utilization|        free_memory|     session_count|
+-------+-------------------+------------------+-------------------+-------------------+------------------+
|  count|             500000|            500000|             500000|             500000|            500000|
|   mean|               NULL|             124.5| 0.6205177399999797|0.37912809999999125|          69.59616|
| stddev|               NULL|14.430884120552715|0.15875173872913106|0.15830931278376184|14.850676696352798|
|    min|03/05/2019 08:06:14|               100|               0.22|                0.0|                32|
|    max|04/09/2019 01:22:46|               149|                1.0|               0.78|               105|
+-------+-------------------+------------------+-------------------+-------------------+------------------+



`.describe()` actually produces another DataFrame with summary statistics about the DataFrame. For example, in this case, we see that there are several columns; there is a summary column, followed by the name of a column in the original DataFrame. For each of those columns in the original DataFrame, we have the same statistics that are calculated.
Using `.describe()`  is an excellent way to get a high-level view of what a data set might be like.

Another statistics we often want to know, is there a correlation between two of the variables?

In [97]:
df_util.printSchema()

root
 |-- event_datetime: string (nullable = true)
 |-- server_id: integer (nullable = true)
 |-- cpu_utilization: double (nullable = true)
 |-- free_memory: double (nullable = true)
 |-- session_count: integer (nullable = true)



In [98]:
df_util.stat.corr('cpu_utilization', 'free_memory')

-0.47047715730806805

In [99]:
df_util.stat.corr('session_count','free_memory')

-0.5008320848876696

Sometimes we want to know how frequent are some items, what are the most frequently occurring items?

There is a method called `.freq()` items for frequent items, which we can use with the DataFrame.

In [100]:
df_freqItems = df_util.stat.freqItems(['server_id'])

In [101]:
df_util.stat.freqItems(['server_id','session_count']).show()

+--------------------+-----------------------+
| server_id_freqItems|session_count_freqItems|
+--------------------+-----------------------+
|[146, 137, 101, 1...|   [92, 101, 83, 104...|
+--------------------+-----------------------+



We can create a result-set that shows some basic statistics for one of the columns by using Spark SQL. Let's do it.

In [102]:
spark.sql("SELECT min(cpu_utilization),\
            max(cpu_utilization),stddev(cpu_utilization) FROM utilization").show()

+--------------------+--------------------+-----------------------+
|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+--------------------+--------------------+-----------------------+
|                0.22|                 1.0|    0.15875173872913106|
+--------------------+--------------------+-----------------------+



And if we want to group the result-set by `server_id`, we can write the following query.

In [103]:
spark.sql("SELECT server_id, min(cpu_utilization),max(cpu_utilization),stddev(cpu_utilization) \
           FROM utilization \
           GROUP BY server_id").show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      108|                0.55|                0.95|    0.11563100171171926|
|      101|                 0.6|                 1.0|    0.11651726263197697|
|      115|                0.44|                0.84|    0.11569664615014985|
|      126|                0.48|                0.88|    0.11542612970702051|
|      103|                0.56|                0.96|    0.11617507884178278|
|      128|                0.38|                0.78|     0.1153254132405078|
|      122|                0.43|                0.83|    0.11563104329209034|
|      111|                0.36|                0.76|    0.11530221569464506|
|      120|                0.35|                0.75|    0.11586355920838642|
|      117|                0.38|                0.78|    0.11534

Now, we are going to calculate statistics on buckets or histograms of data. The idea is, rather than look at each server individually, Spark buckets values according to how frequently they occur in certain ranges. So if we want to know how often does a CPU utilization fall in the range of 1-10 or 11-20 or 21-30, all the way up to 90-91, we could put each of those CPU utilization measures into its bucket and count how many times a CPU utilization goes into that bucket. Let's do it.

In [104]:
spark.sql("SELECT server_id, FLOOR(cpu_utilization*100/10) as Bucket FROM utilization").show()

+---------+------+
|server_id|Bucket|
+---------+------+
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     4|
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     6|
|      100|     6|
|      100|     5|
|      100|     2|
|      100|     4|
|      100|     4|
|      100|     6|
|      100|     4|
|      100|     5|
+---------+------+
only showing top 20 rows



So far, what we have done is we have listed for each server in what  CPU utilization bucket falls at a particular time. Now we want to see how often does a CPU utilization falls into one of those ten buckets.

In [105]:
spark.sql("SELECT count(*), FLOOR(cpu_utilization*100/10) as Bucket \
           FROM utilization GROUP BY Bucket ORDER BY Bucket").show()

+--------+------+
|count(1)|Bucket|
+--------+------+
|    8186|     2|
|   37029|     3|
|   68046|     4|
|  104910|     5|
|  116725|     6|
|   88242|     7|
|   56598|     8|
|   20207|     9|
|      57|    10|
+--------+------+



## Timeseries Analysis

In this section, we are going to work with timeseries data, and timeseries data has a set of measures and a timestamp associated with them. First, let's take a look at utilization table again.

In [106]:
spark.sql("SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
           FROM utilization \
           GROUP BY server_id").show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      108|                0.55|                0.95|    0.11563100171171926|
|      101|                 0.6|                 1.0|    0.11651726263197697|
|      115|                0.44|                0.84|    0.11569664615014985|
|      126|                0.48|                0.88|    0.11542612970702051|
|      103|                0.56|                0.96|    0.11617507884178278|
|      128|                0.38|                0.78|     0.1153254132405078|
|      122|                0.43|                0.83|    0.11563104329209034|
|      111|                0.36|                0.76|    0.11530221569464506|
|      120|                0.35|                0.75|    0.11586355920838642|
|      117|                0.38|                0.78|    0.11534

Sometimes we might want to compare a value within a group. For example, we would like to compare the current CPU utilization for a server to the average CPU utilization of just that server, not the entire population.

We can do that using a windowing function, and in SQL, the windowing functions are specified using an `OVER...PARTITION BY` statement. So let's take a look at how to use that.

In [107]:
spark.sql('SELECT event_datetime, server_id, cpu_utilization, \
           avg(cpu_utilization) OVER (PARTITION BY server_id) as avg_server_util \
           FROM Utilization').show()

+-------------------+---------+---------------+-----------------+
|     event_datetime|server_id|cpu_utilization|  avg_server_util|
+-------------------+---------+---------------+-----------------+
|03/05/2019 08:06:14|      100|           0.57|0.467506000000003|
|03/05/2019 08:11:14|      100|           0.47|0.467506000000003|
|03/05/2019 08:16:14|      100|           0.56|0.467506000000003|
|03/05/2019 08:21:14|      100|           0.57|0.467506000000003|
|03/05/2019 08:26:14|      100|           0.35|0.467506000000003|
|03/05/2019 08:31:14|      100|           0.41|0.467506000000003|
|03/05/2019 08:36:14|      100|           0.57|0.467506000000003|
|03/05/2019 08:41:14|      100|           0.41|0.467506000000003|
|03/05/2019 08:46:14|      100|           0.53|0.467506000000003|
|03/05/2019 08:51:14|      100|           0.51|0.467506000000003|
|03/05/2019 08:56:14|      100|           0.32|0.467506000000003|
|03/05/2019 09:01:14|      100|           0.62|0.467506000000003|
|03/05/201

Now, we have different timestamps for each server ID, different CPU utilization at those particular times, but in this piece of result-set, the average server utilization is always 0.7153 for server ID 112.

Now, we want to calculate the difference any one of these measurements of CPU utilization from the average of that server is?

In [ ]:
spark.sql('SELECT event_datetime, server_id, cpu_utilization, \
           avg(cpu_utilization) OVER (PARTITION BY server_id) as avg_server_util, \
           cpu_utilization - avg(cpu_utilization) OVER (PARTITION BY server_id) as delta_server_util \
           FROM Utilization').show()

That is one of the operations that we can do with the windowing functions. We can compare a particular value in a row to a value of some aggregate function applied to a sub-set of rows.

Another operation that we can do with windowing functions is looking around the neighborhood of a row. For example, we might want to calculate in a sliding window, look at the last three values and average them or look at the previous value, current value, next value, and average them. Let's do it.

In [ ]:
spark.sql('SELECT event_datetime, server_id, cpu_utilization, \
           avg(cpu_utilization) OVER (PARTITION BY server_id ORDER BY event_datetime \
                                       ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) as avg_server_util \
           FROM Utilization').show()

#### Great Job!